In [168]:
ind = {'a': 0,
            't': 1,
            'g': 2,
            'c': 3,
            '_': 4,}

# weight matrix

W = [[ 1, -1, -1, -1, -1],
     [-1,  1, -1, -1, -1],
     [-1, -1,  1, -1, -1],
     [-1, -1, -1,  1, -1],
     [-1, -1, -1, -1,  1],]

In [169]:
def local_alignment(seq1, seq2):
    n = len(seq1)
    m = len(seq2)

    M = [[0 for _ in range(n + 1)] for _ in range(m + 1)]

    M[0] = [0 for i in range(n + 1)]

    for j in range(m + 1):
        M[j][0] = 0
    
    max_val = None
    max_i = None
    max_j = None

    for i in range(1, m + 1):  # seq2
        for j in range(1, n + 1):  # seq1        

            s1 = M[i-1][j-1] + W[ind[seq2[i - 1]]][ind[seq1[j - 1]]]
            s2 = M[i-1][j] + W[ind[seq2[i - 1]]][ind['_']]
            s3 = M[i][j-1] + W[ind['_']][ind[seq1[j - 1]]]

            s = max(max(max(s1, s2), s3), 0)
            M[i][j] = s
            
            if max_val is None or s >= max_val:
                max_val = s
                max_i = i
                max_j = j
    
    seq_new1 = ''
    seq_new2 = ''
    
    cur_i = max_i
    cur_j = max_j
    
    while M[cur_i][cur_j] != 0:
        if M[cur_i - 1][cur_j - 1] >= M[cur_i][cur_j - 1] and M[cur_i - 1][cur_j - 1] >= M[cur_i - 1][cur_j]:
            seq_new1 = seq1[cur_j - 1] + seq_new1
            seq_new2 = seq2[cur_i - 1] + seq_new2
            cur_i = cur_i - 1
            cur_j = cur_j - 1
        elif M[cur_i][cur_j - 1] > M[cur_i - 1][cur_j - 1] and M[cur_i][cur_j - 1] > M[cur_i - 1][cur_j]:
            seq_new1 = seq1[cur_j - 1] + seq_new1
            seq_new2 = '_' + seq_new2
            cur_j = cur_j - 1
        elif M[cur_i - 1][cur_j] > M[cur_i - 1][cur_j - 1] and M[cur_i - 1][cur_j] > M[cur_i][cur_j - 1]:
            seq_new1 = '_' + seq_new1
            seq_new2 = seq2[cur_i - 1] + seq_new2
            cur_i = cur_i - 1
    
    gap_pre_1 = ''
    gap_pre_2 = ''
    gap_post_1 = ''
    gap_post_2 = ''
    
    if cur_j > cur_i:
        gap_pre_2 = '-' * (cur_j - cur_i)
    elif cur_i > cur_j:
        gap_pre_1 = '-' * (cur_i - cur_j)
            
    if len(seq1) - max_j > len(seq2) - max_i:
        gap_post_2 = '-' * ((len(seq1) - max_j) - (len(seq2) - max_i))
    elif len(seq2) - max_i > len(seq1) - max_j:
        gap_post_1 = '-' * ((len(seq2) - max_i) - (len(seq1) - max_j))
        
    seq_new1 = gap_pre_1 + seq1[:cur_j] + '|' + seq_new1 + '|' + seq1[max_j:] + gap_post_1
    seq_new2 = gap_pre_2 + seq2[:cur_i] + '|' + seq_new2 + '|' + seq2[max_i:] + gap_post_2
    
    print(seq_new1, seq_new2, sep='\n')

In [170]:
sequence1, sequence2 = 'actgag agctact'.split()

local_alignment(sequence1, sequence2)

----|act|gag
agct|act|---


Здесь вертикальными линиями выделяем участок локального выравнивания для наглядности

Теперь проверим тоже самое для глобального выравнивания

In [171]:
# global alignment for comparison

def global_alignment(seq1, seq2):
    n = len(seq1)
    m = len(seq2)

    M = [[0 for _ in range(n + 1)] for _ in range(m + 1)]

    M[0] = [-i for i in range(n + 1)]

    for j in range(m + 1):
        M[j][0] = -j

    for i in range(1, m + 1):  # seq2
        for j in range(1, n + 1):  # seq1        

            s1 = M[i-1][j-1] + W[ind[seq2[i - 1]]][ind[seq1[j - 1]]]
            s2 = M[i-1][j] + W[ind[seq2[i - 1]]][ind['_']]
            s3 = M[i][j-1] + W[ind['_']][ind[seq1[j - 1]]]

            s = max(max(s1, s2), s3)
            M[i][j] = s

    seq_new1 = ''
    seq_new2 = ''

    cur_i = m
    cur_j = n

    while cur_i > 0 or cur_j > 0:
        if M[cur_i - 1][cur_j - 1] >= M[cur_i][cur_j - 1] and M[cur_i - 1][cur_j - 1] >= M[cur_i - 1][cur_j]:
            seq_new1 = seq1[cur_j - 1] + seq_new1
            seq_new2 = seq2[cur_i - 1] + seq_new2
            cur_i = cur_i - 1
            cur_j = cur_j - 1
        elif M[cur_i][cur_j - 1] > M[cur_i - 1][cur_j - 1] and M[cur_i][cur_j - 1] > M[cur_i - 1][cur_j]:
            seq_new1 = seq1[cur_j - 1] + seq_new1
            seq_new2 = '_' + seq_new2
            cur_j = cur_j - 1
        elif M[cur_i - 1][cur_j] > M[cur_i - 1][cur_j - 1] and M[cur_i - 1][cur_j] > M[cur_i][cur_j - 1]:
            seq_new1 = '_' + seq_new1
            seq_new2 = seq2[cur_i - 1] + seq_new2
            cur_i = cur_i - 1

    print(seq_new1, seq_new2, sep='\n')

In [172]:
global_alignment(sequence1, sequence2)

a_ctga_g
agct_act


Участок локального выравнивания:

act

act

У глобального отличается